# Model Training & Experimentation Framework

This notebook implements the "Experiment Factory" for the Headway Prediction model. 
It is designed to support the ablation analysis defined in the project abstract, allowing us to vary:
1.  **Lookback Window ($L$):** 30, 45, 60 minutes.
2.  **Input Features:** With or without Terminal Headways ($T$).
3.  **Prediction Horizon:** Recursive prediction up to 60 minutes.

We start by importing the necessary libraries, including TensorFlow/Keras for the Deep Learning components.

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

# set random seeds for reproduceability
np.random.seed(42)
tf.random.set_seed(42)


/Users/danherman/Desktop/headway-prediction/.venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# 1. Configuration class 
# We define an `ExperimentConfig` class to encapsulate all hyperparameters. This makes it easy to switch between different experimental setups (e.g., changing the lookback window or enabling/disabling terminal headways) without rewriting code. 

class ExperimentConfig:
    def __init__(
        self,
        lookback_mins=60,
        forecast_mins=30, 
        time_bin_size_min=5,
        use_terminal_headway=True,
        batch_size=32,
        epochs=32, 
        learning_rate=0.001
    ):
        self.lookback_mins = lookback_mins
        self.forecast_mins = forecast_mins
        self.time_bin_size_min = time_bin_size_min
        self.use_terminal_headway = use_terminal_headway
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate

        # calculated properties
        self.lookback_bins = lookback_mins // time_bin_size_min
        self.forecast_bins = forecast_mins // time_bin_size_min

    def __repr__(self):
        return (f"ExperimentalConfig(L={self.lookback_mins}m, "
                f"F={self.forecast_mins}m, "
                f"Use_T={self.use_terminal_headway}")

# create baseline configuration (exp-A1)
config = ExperimentConfig(
    lookback_mins=30, # Baseline from Abstract
    forecast_mins=15, # single-step target for recursive prediction
    use_terminal_headway=True
)

print(f"Active Configuration {config}")
print(f"Lookback Bins: {config.lookback_bins}")
print(f"Forecast Bins: {config.forecast_bins}")

Active Configuration ExperimentalConfig(L=30m, F=15m, Use_T=True
Lookback Bins: 6
Forecast Bins: 3


## 2. Data Loading & Preparation

We load the preprocessed matrix and schedule data. We then use the `create_dataset` function (adapted from the EDA notebook) to generate the tensors based on the active `config`.

In [3]:
# file paths
MATRIX_PATH = "../data/headway_matrix_full.npy"
SCHEDULE_PATH = "../data/target_terminal_headways.csv"
GLOBAL_START_TIME = "2025-06-06 00:00:00"

def load_and_process_data(config):
    """
    loads raw data and prepares the (X, T, Y) tensors based on the config
    """
    print("Loading data...")
    
    # 1 load matrix
    matrix = np.load(MATRIX_PATH)

    # 2 load and align schedule
    schedule_df = pd.read_csv(SCHEDULE_PATH)
    schedule_df['datetime'] = pd.to_datetime(schedule_df['service_date']) + \
                              pd.to_timedelta(schedule_df['departure_seconds'], unit='s')
    schedule_df = schedule_df.set_index('datetime').sort_index()
    schedule_df = schedule_df[~schedule_df.index.duplicated(keep='first')]
    schedule_df = schedule_df[schedule_df.index >= GLOBAL_START_TIME]

    # fill nans and resample
    schedule_df['scheduled_headway_min'] = schedule_df['scheduled_headway_min'].bfill()
    time_coords = pd.date_range(start=GLOBAL_START_TIME, periods=matrix.shape[0], freq=f"{config.time_bin_size_min}min")
    schedule_resampled = schedule_df['scheduled_headway_min'].resample(f'{config.time_bin_size_min}min').ffill()
    schedule_aligned = schedule_resampled.reindex(time_coords, method='ffill').bfill().values

    # 3. create tnesors
    print(f"Generating tensors with L={config.lookback_bins} bins, F={config.forecast_bins} bins...")
    X, T, Y = [], [], []

    for i in range(config.lookback_bins, len(matrix) - config.forecast_bins):
        # Input X: Past L steps
        X.append(matrix[i-config.lookback_bins:i, :])
        # Input T: Future F steps
        T.append(schedule_aligned[i:i+config.forecast_bins])
        # Target Y: Future F steps
        Y.append(matrix[i:i+config.forecast_bins, :])

    X = np.array(X)[..., np.newaxis] #(Batch, Time, Space, 1)
    T = np.array(T)[..., np.newaxis] #(Batch, Time, 1)
    Y = np.array(Y)[..., np.newaxis] #(Batch, Time, Space, 1)

    return X, T, Y

# execute data loading
X, T, Y = load_and_process_data(config)

print(f"\nData Shapes:")
print(f"X (Context): {X.shape}")
print(f"T (Intent): {T.shape}")
print(f"Y (Target): {Y.shape}")

Loading data...
Generating tensors with L=6 bins, F=3 bins...

Data Shapes:
X (Context): (52829, 6, 160, 1)
T (Intent): (52829, 3, 1)
Y (Target): (52829, 3, 160, 1)
